In [40]:
import os
import rasterio
import numpy as np
import pandas as pd
import re

This code generates the annual median nighttime lights intensity from VIIRS monthly images.

In [41]:
# ==========================================
# 1. SETUP
# ==========================================
base_dir = "/Users/ruben/Desktop/Thesis/TrainingData/VIIRS-PhilSA-50Sample"
output_csv = "viirs_ntl_labels_sample50.csv"

results = []
skipped_folders = []

print(f"Scanning {base_dir}...")

Scanning /Users/ruben/Desktop/Thesis/TrainingData/VIIRS-PhilSA-50Sample...


In [42]:
# ==========================================
# 2. ROBUST PROCESSING LOOP
# ==========================================
all_folders = [f for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f))]

for folder_name in sorted(all_folders):
    cluster_path = os.path.join(base_dir, folder_name)
    
    # 1. Extract ID (Flexible Regex)
    # Matches 'VIIRS_PH2022' followed by ANY digits at the end
    match = re.search(r'VIIRS_PH20220*(\d+)$', folder_name)
    if not match:
        print(f"Skipping {folder_name}: No ID found.")
        continue
    
    cluster_id = int(match.group(1))
    
    # 2. Process TIFs
    # Read all TIFs in this folder with better handling
    cluster_monthly_avgs = []
    
    tif_files = [f for f in os.listdir(cluster_path) if f.endswith(".tif")]
    print(f"  Found {len(tif_files)} TIFF files")

    if len(tif_files) == 0:
        skipped_folders.append({"folder": folder_name, "cluster": cluster_id, "reason": "No TIFF files"})
        continue
    
    for file in tif_files:
        file_path = os.path.join(cluster_path, file)
        try:
            with rasterio.open(file_path) as src:
               
                data = src.read(1)
                
               # DEBUG: Check basic stats
                #print(f"    File: {file[:30]}... Shape: {data.shape}")
                #print(f"      Raw - Min: {data.min():.3f}, Max: {data.max():.3f}, Mean: {data.mean():.3f}")

                # Get valid data based on nodata value
                nodata = src.nodata
                if nodata is not None:
                    # Create mask for valid data (not equal to nodata)
                    valid_mask = (data != nodata)
                    valid_data = data[valid_mask]
                    print(f"Using nodata value: {nodata}")
                else:
                    # If no nodata defined, use a threshold approach
                    # VIIRS often uses -1.5 as background
                    valid_mask = (data > -1.5)
                    valid_data = data[valid_mask]
                    print(f"No nodata value, using > -1.5 threshold")
                
                # Check if we have any valid pixels
                if len(valid_data) == 0:
                    print(f"No valid pixels found")
                    continue
                
                # Calculate average of valid pixels
                # Using median is more robust to outliers
                avg_val = np.median(valid_data)
                
                # Print stats about valid data
                print(f"      Valid pixels: {len(valid_data)}/{data.size} ({len(valid_data)/data.size*100:.1f}%)")
                print(f"      Valid - Min: {valid_data.min():.3f}, Max: {valid_data.max():.3f}, Median: {avg_val:.3f}")
                
                # More lenient threshold: only filter if all valid data is extremely negative
                if avg_val > -10:  # Allow slightly negative values
                    cluster_monthly_avgs.append(avg_val)
                    print(f"Added to monthly averages")
                else:
                    print(f"Skipped (median too negative): {avg_val:.3f}")
                    
        except Exception as e:
            print(f"    ✗ Error reading {file}: {e}")
            continue

    # 3. Save Result
    # Calculate annual median
    if cluster_monthly_avgs:
        annual_median = np.median(cluster_monthly_avgs)
        print(f"Annual median: {annual_median:.3f} (from {len(cluster_monthly_avgs)} months)")
        
        results.append({
            'DHSCLUST': cluster_id,
            'NTL_Value': annual_median,
            #'months_used': len(cluster_monthly_avgs)  # Track how many months contributed
        })
    else:
        print(f"  ✗ No valid monthly averages")
        skipped_folders.append({
            "folder": folder_name, 
            "cluster": cluster_id, 
            "reason": "No valid data after filtering"
        })
    
    print("-" * 50)

  Found 12 TIFF files
No nodata value, using > -1.5 threshold
      Valid pixels: 529/529 (100.0%)
      Valid - Min: 0.000, Max: 0.910, Median: 0.350
Added to monthly averages
No nodata value, using > -1.5 threshold
      Valid pixels: 529/529 (100.0%)
      Valid - Min: -0.220, Max: 1.070, Median: 0.220
Added to monthly averages
No nodata value, using > -1.5 threshold
      Valid pixels: 529/529 (100.0%)
      Valid - Min: -0.040, Max: 0.750, Median: 0.290
Added to monthly averages
No nodata value, using > -1.5 threshold
      Valid pixels: 529/529 (100.0%)
      Valid - Min: -0.080, Max: 0.670, Median: 0.000
Added to monthly averages
No nodata value, using > -1.5 threshold
      Valid pixels: 529/529 (100.0%)
      Valid - Min: 0.000, Max: 0.700, Median: 0.330
Added to monthly averages
No nodata value, using > -1.5 threshold
      Valid pixels: 529/529 (100.0%)
      Valid - Min: 0.000, Max: 1.250, Median: 0.390
Added to monthly averages
No nodata value, using > -1.5 threshold
     

In [43]:
# Final summary
print("\n" + "="*50)
print("PROCESSING SUMMARY")
print("="*50)

if len(results) > 0:
    df = pd.DataFrame(results)
    
    # Create classes
    df['NTL_Class'] = pd.qcut(df['NTL_Value'], q=3, labels=[0, 1, 2])
    
    # Save to CSV
    df.to_csv(output_csv, index=False)
    
    print(f"✓ Successfully processed: {len(df)} clusters")
   # print(f"✓ Months used per cluster: {df['months_used'].describe().to_dict()}")
    print(f"✓ Class Distribution:\n{df['NTL_Class'].value_counts().sort_index()}")
    print(f"✓ Labels saved to: {output_csv}")
    
else:
    print("✗ No data found in any folder")

# Show skipped folders
if skipped_folders:
    print("\n" + "-"*50)
    print("SKIPPED FOLDERS:")
    for skip in skipped_folders:
        print(f"  • {skip}")
    
    # Save skipped folders to a separate CSV for review
    skip_df = pd.DataFrame(skipped_folders)
    skip_df.to_csv("skipped_clusters.csv", index=False)
    print(f"\nDetailed skip log saved to: skipped_clusters.csv")


PROCESSING SUMMARY
✓ Successfully processed: 50 clusters
✓ Class Distribution:
NTL_Class
0    17
1    16
2    17
Name: count, dtype: int64
✓ Labels saved to: viirs_ntl_labels_sample50.csv
